In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import OneHotEncoder

import pickle

In [14]:
throws = pd.read_csv("../data/throws_clean.csv")


# print(throws.head())
throw_types = pd.get_dummies(throws.throw_type).rename(columns = {"BH":"backhand","F":"forehand","H":"hammer"})
print(throws.columns)
throw_analysis = pd.concat([throws,throw_types], axis = 1)[["Xdist","Ydist","backhand","forehand","hammer","completion"]].dropna()

throw_analysis = throws[["Xdist","Ydist","Stall","completion"]].dropna()
throw_analysis["Ydist"] = abs(throw_analysis["Ydist"])
# throws = pd.concat(throws)

# print(throw_types)


throw_analysis.shape

Index(['ThrowID', 'X2', 'GameID', 'throw_team_score', 'def_team_score',
       'Score', 'point_per_game', 'poss_per_point', 'score_on_poss',
       'Direction', 'throw_per_game', 'throw_per_point', 'throw_per_poss',
       'throw_team', 'Thrower', 'receiver', 'hand_rl', 'throw_type',
       'throw_curve', 'Force', 'Stall', 'completion', 'bad_throw',
       'GEO_Xstart', 'GEO_Ystart', 'GEO_Xend', 'GEO_Yend', 'RO_Xstart',
       'RO_Ystart', 'RO_Xend', 'RO_Yend', 'Xdist', 'Ydist', 'Dist', 'pull',
       'goal'],
      dtype='object')


(2046, 4)

In [10]:
def split_data(data, train_share = 0.8, response_col = "completion", random_state = 0):
    feature_cols = data.drop(columns = response_col)
    response = data[[response_col]]
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(feature_cols, response,
                                                                                train_size=train_share,
                                                                                random_state=random_state)
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = split_data(throw_analysis)

print(y_train.head())
print(X_train.head())

      completion
83           1.0
1621         1.0
1243         1.0
1579         1.0
329          1.0
      Xdist  Ydist  Stall
83    -15.0    2.0    7.0
1621   11.0   14.0    0.0
1243    9.0   12.0    2.0
1579   10.0   13.0    1.0
329    20.0    6.0    3.0


In [17]:
def train_model(X_train, X_test, y_train, y_test):
    logistic = LogisticRegressionCV(cv = 10).fit(X_train, y_train)
    
    
    print(logistic.coef_)
    
    out = logistic.predict_proba(X_test)
    return logistic

model = train_model(X_train, X_test, y_train, y_test)

model.C_

C:\Users\tonyc\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[-0.04170647 -0.00220392 -0.00520637]]


array([0.0001])

In [33]:
user_input = pd.DataFrame(data={ "Xdist":[30.0], "Ydist":[-5.0], "Stall":[4]})

complete = pd.Series(model.predict_proba(X_train)[:,1], name="comp_perc")
preds = pd.concat([throw_analysis, complete], axis = 1)

preds["comp"] = preds.comp_perc > 0.9

preds.comp.mean()

0.7412280701754386